# Análise de Indicadores Socioeconômicos e Votação (Brasil 2022)
Este notebook carrega e combina:
1. **Indicadores socioeconômicos por Unidade da Federação (UF) em 2022**
2. **Resultado de votos nominais válidos do 1º turno das eleições de 2022**, agregados por UF e **agrupados por ideologia partidária**.

O resultado final é salvo em um CSV (`uf_votos_ideologia_socioeco_2022.csv`).

In [1]:
import pandas as pd
from pathlib import Path

## Definição dos caminhos dos arquivos de entrada
Ajuste conforme seu diretório:

In [ ]:
BASE = Path('.')  # ajuste se necessário

# Arquivos de entrada
path_indicadores = BASE / '/home/fbotero/Documents/POS/data_lake/raw/csv/indicadores_uf_2022.csv'
path_gini         = BASE / '/home/fbotero/Documents/POS/data_lake/raw/csv/gini_uf_2022.csv'
path_analf        = BASE / '/home/fbotero/Documents/POS/data_lake/raw/csv/analfabetismo_uf_2022.csv'
path_votos        = BASE / '/home/fbotero/Documents/POS/data_lake/raw/csv/votacao_partido_munzona_2022_BR.csv'

# Saída
out_csv           = BASE / '/home/fbotero/Documents/POS/data_lake/processed/uf_votos_ideologia_socioeco_2022.csv'

## 1. Carregar e padronizar indicadores socioeconômicos

In [ ]:
ind = pd.read_csv(path_indicadores)

# Renomear e ajustar colunas
ind.rename(columns={'PIB_R$mi': 'PIB_milhoes',
                    'PIB_per_capita_R$': 'PIB_per_capita',
                    'Esperanca_vida': 'Esperanca_vida_anos',
                    'Mort_infantil_per_mil': 'Mort_infantil_pmil',
                    'Analfabetismo_%': 'Analfabetismo_perc',
                    'IPCA_2022_%': 'IPCA_2022_perc'}, inplace=True)

# Completar Gini, caso não exista
if 'Gini' not in ind.columns:
    gini = pd.read_csv(path_gini)
    ind = ind.merge(gini[['UF', '2022']].rename(columns={'2022': 'Gini'}), on='UF', how='left')

# Completar Analfabetismo, caso ausente
if 'Analfabetismo_perc' not in ind.columns:
    analf = pd.read_csv(path_analf)
    ind = ind.merge(analf[['UF', '2022']].rename(columns={'2022': 'Analfabetismo_perc'}), on='UF', how='left')

ind.head()

## 2. Carregar votos nominais e atribuir ideologia partidária

In [ ]:
votes = pd.read_csv(path_votos, sep=';', encoding='latin-1',
                     usecols=['NR_TURNO', 'SG_UF', 'SG_PARTIDO', 'QT_VOTOS_NOMINAIS_VALIDOS'])

# Filtra 1º turno e votos > 0
votes = votes[(votes['NR_TURNO'] == 1) & (votes['QT_VOTOS_NOMINAIS_VALIDOS'] > 0)]

# Mapeamento partido -> ideologia
ideology_map = {
    'PT':'esquerda','PCB':'esquerda','UP':'esquerda','PSTU':'esquerda',
    'PDT':'centro-esquerda','PSB':'centro-esquerda','REDE':'centro-esquerda','PV':'centro-esquerda',
    'MDB':'centro','PSD':'centro','PODE':'centro','CIDADANIA':'centro','AVANTE':'centro',
    'UNIÃO':'centro-direita','PSDB':'centro-direita','PP':'centro-direita','NOVO':'centro-direita',
    'SOLIDARIEDADE':'centro-direita',
    'PL':'direita','PTB':'direita','DC':'direita','PATRIOTA':'direita','PRTB':'direita'
}

votes['ideologia'] = votes['SG_PARTIDO'].map(ideology_map).fillna('outros')
votes.head()

## 3. Agregar votos por UF e ideologia

In [ ]:
agg = (votes.groupby(['SG_UF','ideologia'])['QT_VOTOS_NOMINAIS_VALIDOS']
         .sum().unstack(fill_value=0).reset_index().rename(columns={'SG_UF':'UF'}))

# Calcula participação percentual
ideology_cols = [c for c in agg.columns if c != 'UF']
agg['total_votos'] = agg[ideology_cols].sum(axis=1)
for col in ideology_cols:
    agg[col + '_perc'] = (agg[col] / agg['total_votos']) * 100

agg.head()

## 4. Combinar indicadores e votação

In [ ]:
combined = ind.merge(agg, on='UF', how='left')
combined.head()

## 5. Salvar conjunto combinado em CSV

In [ ]:
combined.to_csv(out_csv, index=False)
print(f'Arquivo salvo em: {out_csv.resolve()}')

### Pronto!
Agora você tem um dataset unificado `uf_votos_ideologia_socioeco_2022.csv` com indicadores socioeconômicos e votação por bloco ideológico. A partir daqui, você pode seguir para análises estatísticas, visualizações ou modelos de ML.